In [16]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from datetime import datetime
import shap
import json
import joblib
from pycaret.classification import *

In [17]:
path = 'c:\\Users\\wongi\\Desktop\\알고리즘랩스\\프로젝트\\현대산업개발\\develop'

In [18]:
raw_data = pd.read_csv(f'{path}\\algorithmlabs.inference\\inference_data_promotion.csv')
model = joblib.load(f'{path}\\algorithmlabs.inference\\pickle_promotion\\model.pkl')
explainer = joblib.load(f'{path}\\algorithmlabs.inference\\pickle_promotion\\explainer.pkl')
encoder = joblib.load(f'{path}\\algorithmlabs.inference\\pickle_promotion\\label_encoder.pkl')
name_dict = joblib.load(f'{path}\\algorithmlabs.inference\\pickle_promotion\\name_dict.pkl')

In [19]:
raw_data['NO_POINT_SUM'] = round(raw_data['NO_POINT_SUM'], 1)
inf_data = raw_data.copy()

In [20]:
inf_data['YN_HANDICAP'].replace({'Y' : 1, 'N' : 0}, inplace=True)
inf_data['YN_SUPPORTED'].replace({'Y' : 1, 'N' : 0}, inplace=True)
inf_data['YN_FOREIGNER'].replace({'Y' : 1, 'N' : 0}, inplace=True)
inf_data['YN_GENDER'].replace({'M' : 1, 'W' : 0}, inplace=True)

In [21]:
for label in np.unique(inf_data['DS_BIRTHPLACE']):
    if label not in encoder.classes_: 
        encoder.classes_ = np.append(encoder.classes_, label) 
inf_data['DS_BIRTHPLACE'] = encoder.transform(inf_data['DS_BIRTHPLACE'])

In [22]:
inf_data.drop(['ID_SABUN', 'DS_BONBU', 'DS_DEPT', 'DS_JOBFAMILY', 'DS_ZONE', 'DS_JOBTYPE', 'DS_DUTY', 'DS_ADOPTYPE', 'DS_JOBFAMILY_JOIN',
               'DS_HNAME', 'DS_POSITION', 'DS_OCCUPATION', 'DS_HANDPHONE', 'DS_EMAIL', 'DT_JOIN', 'DT_GROUP_JOIN'], axis=1, inplace=True)

In [23]:
predict_proba = model.predict_proba(inf_data)[:,1]
raw_data['predict_proba'] = predict_proba

In [24]:
raw_data = raw_data[raw_data.columns[1:2].tolist() + raw_data.columns[-1:].tolist() + raw_data.columns[0:1].tolist() + raw_data.columns[2:-1].tolist()]

In [25]:
columns = ['ID_SABUN', 'predict_proba', 'DT_YEAR', 'DS_HNAME', 'DS_POSITION', 'DS_OCCUPATION', 'DS_HANDPHONE', 'DS_EMAIL', 'CD_BONBU', 'CD_DEPT',
       'CD_JOBFAMILY', 'DS_BONBU', 'DS_DEPT', 'DS_JOBFAMILY', 'DT_BIRTH', 'DS_BIRTHPLACE', 'DT_GROUP_JOIN', 'DT_JOIN', 'NO_POSITION_YEAR', 'NO_POINT_SUM',
       'NO_POINT_LANGUAGE', 'NO_POINT_LICENSE', 'NO_POINT_REWARD', 'NO_ADJUST_SUM', 'CD_ZONE',
       'DS_ZONE', 'CD_JOBTYPE', 'CD_DUTY', 'CD_ADOPTYPE', 'YN_HANDICAP', 'YN_SUPPORTED', 'YN_FOREIGNER', 'YN_GENDER', 'CD_JOBFAMILY_JOIN', 
       'DS_JOBTYPE', 'DS_DUTY', 'DS_ADOPTYPE', 'DS_JOBFAMILY_JOIN', 'DUTY_CNT', 'PROJECT_CNT', 'FINAL_GRADE_THIS_YEAR',
       'FINAL_GRADE_ONE_YEAR', 'FINAL_GRADE_TWO_YEAR', 'MNTR', 'BEFORE_PROMOTION_CNT']
raw_data = raw_data[columns]

In [26]:
raw_data.drop(['CD_BONBU', 'CD_DEPT', 'CD_JOBFAMILY', 'CD_ZONE', 'CD_JOBTYPE', 'CD_DUTY', 'CD_ADOPTYPE', 'CD_JOBFAMILY_JOIN'], axis=1, inplace=True)

In [27]:
shap_values = explainer.shap_values(inf_data, check_additivity=False)
new_shap=[]

if len(shap_values) == 2 :
    for i in range(len(shap_values[1])) :
        new_shap.append(shap_values[1][i]/abs(shap_values[1][i]).sum())
else : 
    for i in range(len(shap_values)) :
        new_shap.append(shap_values[i]/abs(shap_values[i]).sum())
        
feature_weight = pd.DataFrame(new_shap, columns=inf_data.columns)
feature_weight['ID_SABUN'] = raw_data['ID_SABUN']
feature_weight = feature_weight[feature_weight.columns[-1:].tolist() + feature_weight.columns[:-1].tolist()]

In [28]:
for name in raw_data.columns.tolist() : 
    for name_ex in name_dict.keys() : 
        if name == name_ex : 
            raw_data.rename(columns={name : name_dict[name_ex]}, inplace=True)

In [29]:
for name in feature_weight.columns.tolist() : 
    for name_ex in name_dict.keys() : 
        if name == name_ex : 
            feature_weight.rename(columns={name : name_dict[name_ex]}, inplace=True)

In [30]:
raw_data.to_csv(f'{path}\\algorithmlabs.inference\\infer_result_promotion.csv', index=False)
feature_weight.to_csv(f'{path}\\algorithmlabs.inference\\feature_weight_promotion.csv', index=False)